In [ ]:
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
# from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.indexes import VectorstoreIndexCreator
from langchain.chat_models import openai
from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI
import requests
import os
from datetime import datetime
from time import sleep
import pandas as pd
import faiss
from pathlib import Path

In [ ]:
os.environ["OPENAI_KEY"] = "***"
os.environ["AZURE_OPENAI_KEY"] = os.environ["OPENAI_KEY"]
os.environ["AZURE_OPENAI_API_KEY"] = os.environ["OPENAI_KEY"]
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://***.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo"

In [ ]:
client = AzureOpenAI(
  azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-01" #2024-02-15-preview ?
)

message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."}]

completion = client.chat.completions.create(
  model="gpt-35-turbo", # model = "deployment_name"
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

print(completion)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage


llm = AzureChatOpenAI(
    openai_api_version="2024-02-01",
    azure_deployment="gpt-35-turbo",
)

llm4 = AzureChatOpenAI(
    openai_api_version="2024-02-01",
    azure_deployment="gpt-4o",
)

sentence_to_translate = "I want to evaluate ChatGPT's answers."

message = HumanMessage(
    content=f"Translate this sentence from English to French : {sentence_to_translate}"
)
llm.invoke([message])

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

def translate(sentence):

    translate_prompt_template = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant who translates the following sentence from english to french."),
        ("human", "{sentence}"),
    ])

    translate_chain = translate_prompt_template | llm4

    translation = translate_chain.invoke({"sentence": sentence}).content
    # print(translation)
    return translation

In [ ]:
translate(sentence_to_translate)

In [ ]:
# !pip install unstructured
# !pip install "unstructured[pdf]"

In [ ]:
from langchain_community.document_loaders import DirectoryLoader


directory_path = "microsoft_learn"

loader = DirectoryLoader(directory_path)
pages = loader.load_and_split()
pages

# partition_pdf is not available. Install the pdf dependencies with pip install "unstructured[pdf]"

RAG : recherche par similarité (en utilisant les embeddings)

Extract text from pdf 

In [ ]:
#!pip install pdfminer.six

In [ ]:
# from pdfminer.high_level import extract_text


# dirs = os.listdir(directory_path)
# output = "pdftotext.txt"
# # supprimer le fichier s'il existe déjà
# if os.path.exists(output):  
#     os.remove(output) 
# for file in dirs:
#     path_file = os.path.join(directory_path, file)
#     print(f"PDF path file: {path_file}")
#     text = extract_text(path_file)
#     with open(output, 'a', encoding='utf-8') as fichier:
#         fichier.write(text)


In [ ]:
# !pip install pypdf

In [ ]:
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader


# loader = TextLoader("pdftotext.txt", encoding='utf-8')
# loader = PyPDFLoader("microsoft_learn/intelligence_artificielle.pdf")
loader = DirectoryLoader("microsoft_learn/", glob="**/*.pdf", loader_cls=PyPDFLoader)

In [ ]:
# loader multi files types
# https://github.com/langchain-ai/langchain/discussions/18559

In [ ]:
documents= loader.load()
for document in documents:
    document.metadata["filename"] = Path(document.metadata["source"]).name
    document.metadata["loading_date"] = datetime.today().strftime('%Y%m%d')
    document.metadata["user"] = Path(document.metadata["source"]).parent.name
len(documents)

In [ ]:
documents[0].metadata

In [ ]:
documents[0].page_content

In [ ]:
# chunk settings  
chunk_size = 200
chunk_overlap = 30 # duplication des informations entre les vecteurs coupés 

In [ ]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter


# text_splitter = CharacterTextSplitter(
#                     chunk_size=chunk_size,
#                     chunk_overlap=chunk_overlap,
#                     separator="\n",
#                 )

text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=chunk_size,
                    chunk_overlap=chunk_overlap,
                    separators=["\n"],
                )
chunks = text_splitter.split_documents(documents)

nb_chunks = len(chunks)
print(f"Nb of chunks: {nb_chunks}")

In [ ]:
chunks[0]

In [ ]:
print(chunks[nb_chunks-1].metadata["user"])
print(chunks[nb_chunks-1].page_content)

In [ ]:
# # https://stackoverflow.com/questions/76886954/multiple-file-loading-and-embeddings-with-openai

# print("Loading data...")
# pdf_folder_path = "content/"
# print(os.listdir(pdf_folder_path))

# # Load multiple files
# loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]

# print(loaders)

# all_documents = []

# for loader in loaders:
#     print("Loading raw document..." + loader.file_path)
#     raw_documents = loader.load()

#     print("Splitting text...")
#     text_splitter = CharacterTextSplitter(
#         separator="\n\n",
#         chunk_size=800,
#         chunk_overlap=100,
#         length_function=len,
#     )
#     documents = text_splitter.split_documents(raw_documents)
#     all_documents.extend(documents)

# print("Creating vectorstore...")
# embeddings = OpenAIEmbeddings()
# vectorstore = FAISS.from_documents(all_documents, embeddings)

# with open("vectorstore.pkl", "wb") as f:
#     pickle.dump(vectorstore, f)

Enregistrer les chunks en tant que vecteur en utilisant faiss  
pour résoudre le problème des tokens appels nombreux API, on découpe les chunks
 en blocs , sleep 
save le tout en local pour ne pas relancer à chaque fois

In [ ]:
# !pip install -U langchain-community faiss-cpu langchain-openai tiktoken

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import AzureOpenAIEmbeddings
# https://github.com/facebookresearch/faiss/wiki/Special-operations-on-indexes


embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    api_key=os.environ["AZURE_OPENAI_KEY"],
)

CALC_EMBEDDINGS = False
BATCH_EMBEDDINGS = False

if CALC_EMBEDDINGS:
    if BATCH_EMBEDDINGS:

        # Iterate over each chunk of documents
        # https://github.com/langchain-ai/langchain/issues/7343
        # https://stackoverflow.com/questions/76644262/adding-large-chunks-of-text-after-embedding-into-pinecone-without-openai-ratelim
        batch_size = 50

        for i in range(0, len(chunks), batch_size):
            print(f"batch {i} to {i+batch_size}")
            if 'faiss_vectorstore' not in locals() or 'faiss_vectorstore' not in globals():
                faiss_vectorstore = FAISS.from_documents(chunks[i:i+batch_size], embeddings)
            else:
                faiss_vectorstore.add_documents(chunks[i:i+batch_size])
                print(f"...sleep 6s...")
                sleep(6)  # Add delay here
    else:
        faiss_vectorstore = FAISS.from_documents(chunks, embeddings)

    faiss_vectorstore.save_local("faiss_index")
else:
    faiss_vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

faiss_retriever = faiss_vectorstore.as_retriever()

# TODO : optimiser la taille des chunks plus tard
# LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
# The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).

In [ ]:
# import pickle

# #with open("index.pkl", "ab") as f:
# with open("index.pkl", "wb") as f:
#     pickle.dump(faiss_vectorstore, f)
#     f.close()

# # DEBUG : cannot pickle '_thread.RLock' object
# # https://github.com/langchain-ai/langchain/issues/14581

In [ ]:
question = "Qu'est ce que le service Azure OpenAI ?"
question = "Citer trois services cognitifs Azure."
# question = "Ce document est-il autorisé ?"

In [ ]:
searchDocs = faiss_vectorstore.similarity_search(question, filter={"user":"Paul"} )

print(f"Retrieve docs: {len(searchDocs)}")
for i in range(len(searchDocs)):
    print(f"- {i} from {searchDocs[i].metadata['filename']} : \n'''{searchDocs[i].page_content}'''")

Retrievers are used to find relevant documents or passages that contain the answer to a given query.
The system "retrieves" any documents that could be relevant in answering the question, and then passes those documents (along with the original question) to the language model for a "generation" step.

La question est posée en utilisant le chat,
toujours en utilisant la similarité sématique 


Test utilisant LCEL


In [ ]:
prompt_template = """Utilisez les éléments de contexte suivants pour répondre à la question contenue dans les 3 crochets à la fin. Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.
Veuillez fournir une réponse correcte sur le plan factuel et basée sur les informations extraites du magasin de vecteurs.
Veuillez également mentionner toute citation étayant la réponse, si elle est présente dans le contexte fourni, entre deux doubles guillemets "" .
Ne relancez pas la conversation en demandant des précisions sur la question. Répondez exclusivement en français.

CONTEXT:
{context}

QUESTION:```{question}```
REPONSE:
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context","question"]
  )

chain_type_kwargs = {"prompt": PROMPT}

In [ ]:
# prompt_template =  """
#                     Answer the question only according to the context: {context}
#                     Question: {question}
#                     """

# PROMPT = ChatPromptTemplate.from_template(prompt_template)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


retrieval_chain = (
    {"context": faiss_retriever, "question": RunnablePassthrough()}
    | PROMPT
    | llm
    | StrOutputParser()
)

In [ ]:
question = "Qu'est ce que le service Azure OpenAI ?"
question = "Citer trois services cognitifs Azure."

In [ ]:
retrieval_chain.invoke(question)

In [ ]:
from langchain.chains import RetrievalQA


filtered_retriever = faiss_vectorstore.as_retriever(search_kwargs={"filter": {"user":'Paul'}})

qa_stuff = RetrievalQA.from_chain_type( #creer une chain pour répondre à la question
    llm=llm, 
    chain_type="stuff", # tous les documents d'un coup et les passe au LLM
    # chain_type_kwargs=chain_type_kwargs,
    retriever=filtered_retriever, 
    return_source_documents= True,
    verbose=True
)

In [ ]:
# question = "Pourquoi utiliser Azure OpenAI ?"
question = "Quel service utiliser pour de la traduction ?"
# question = "Est-ce que le contenu du contexte est autorisé ?"

result = qa_stuff.invoke(question)
result

Préparer un golden dataset 


In [ ]:
# TODO : translate questions and ground_truth
# https://docs.ragas.io/en/stable/getstarted/testset_generation.html

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# generator with openai models
generator_llm = llm
critic_llm = llm4
# embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

In [ ]:
# generate testset
GENERATE_TESTSET = False

if GENERATE_TESTSET:

    testset = generator.generate_with_langchain_docs(
                            documents,
                            test_size=20,
                            distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25}
                            )
    
    testset_df = testset.to_pandas()
    testset_df['question_en'] = testset_df['question']
    testset_df['question'] = testset_df['question'].apply(translate)
    testset_df.to_csv('testset.csv', index=False)  
else :
    testset_df = pd.read_csv('testset.csv')

In [ ]:
testset_df['question'] = testset_df['question_en'].apply(translate)

In [ ]:
testset_df.head(10)

In [ ]:
(testset_df['ground_truth'] == 'nan').sum()

# Replace "nan" in ground_truth by good answer

In [ ]:
testset_df.iloc[2]['question']

In [ ]:
testset_df.iloc[2]['ground_truth']

In [ ]:
questions = [
"Qu'est-ce que le service Azure OpenAI ?",
"Comment fonctionne le service Azure OpenAI ?",
"Quels sont les avantages du service Azure OpenAI ?",
"Quels sont les cas d'utilisation du service Azure OpenAI ?",
"Comment mettre en œuvre le service Azure OpenAI ?",
"Quels sont les composants principaux du service Azure OpenAI ?",
"Quels sont les modèles d'IA accessibles via le service Azure OpenAI ?",
"Quelles tâches les modèles d'IA du service Azure OpenAI peuvent-ils accomplir ?",
"Quelle est l'infrastructure sur laquelle repose le service Azure OpenAI ?",
"Quels outils le service Azure OpenAI fournit-il pour la formation et le réglage des modèles d'IA ?"
]

answers = [  
"Le service Azure OpenAI est un service cloud qui permet aux développeurs et aux chercheurs d'accéder à la plateforme OpenAI, une initiative qui vise à créer une intelligence artificielle générale (AGI) bénéfique pour l'humanité. Il offre la possibilité d'utiliser des modèles d'IA pré-entraînés ou personnalisés, de les déployer sur le cloud ou en périphérie, et de les intégrer à des applications et des scénarios variés.",   
"Le service Azure OpenAI repose sur deux composants principaux : le portail Azure OpenAI et les API Azure OpenAI. Le portail est une interface web qui permet de créer, de gérer et de surveiller les ressources du service, tandis que les API sont des interfaces de programmation qui permettent d'interagir avec les modèles d'IA du service.",   
"Le service Azure OpenAI présente plusieurs avantages, parmi lesquels l'accès à des modèles d'IA de pointe, une grande flexibilité, une haute performance grâce à l'infrastructure cloud de Microsoft Azure, et une facilité d'intégration grâce à des API simples et des SDK dans différents langages de programmation.",   
"Le service Azure OpenAI peut être utilisé pour de nombreux cas d'utilisation, dans différents domaines et secteurs d'activité, tels que l'éducation, la santé, le divertissement et le commerce.",   
"Pour mettre en œuvre le service Azure OpenAI, il faut créer un compte Microsoft Azure, souscrire à un abonnement au service, créer une clé d'API, choisir ou créer un modèle d'IA, créer un point de terminaison, envoyer des requêtes au point de terminaison, et analyser les résultats des requêtes.",   
"Les composants principaux du service Azure OpenAI sont le portail Azure OpenAI et les API Azure OpenAI.",   
"Les modèles d'IA accessibles via le service Azure OpenAI incluent GPT-3, Codex, DALL-E et CLIP.",   
"Les modèles d'IA du service Azure OpenAI peuvent réaliser des tâches complexes et variées, telles que la génération de texte, la compréhension du langage naturel, la vision par ordinateur, la synthèse vocale ou la création de contenu multimédia.",   
"L'infrastructure sur laquelle repose le service Azure OpenAI est l'infrastructure cloud de Microsoft Azure.",   
"Le service Azure OpenAI fournit des outils de formation et de réglage pour créer son propre modèle d'IA."
]  

ground_truths = [  
["Le service Azure OpenAI est un service cloud qui permet aux développeurs et aux chercheurs d'accéder à la plateforme OpenAI", "Le service Azure OpenAI offre la possibilité d'utiliser des modèles d'IA pré-entraînés ou personnalisés, de les déployer sur le cloud ou en périphérie, et de les intégrer à des applications et des scénarios variés."],  
["Le service Azure OpenAI repose sur deux composants principaux : le portail Azure OpenAI et les API Azure OpenAI.", "Le portail Azure OpenAI est une interface web qui permet de créer, de gérer et de surveiller les ressources du service Azure OpenAI, telles que les abonnements, les clés, les modèles, les points de terminaison et les requêtes."],  
["Le service Azure OpenAI présente plusieurs avantages, parmi lesquels :", "Il garantit une haute performance, en s'appuyant sur l'infrastructure cloud de Microsoft Azure, qui assure une disponibilité, une scalabilité et une sécurité optimales."],  
["Le service Azure OpenAI peut être utilisé pour de nombreux cas d'utilisation, dans différents domaines et secteurs d'activité.", "Dans le domaine du commerce, le service Azure OpenAI peut être utilisé pour créer des outils d'optimisation, tels que des recommandations personnalisées, des prévisions de ventes, des analyses de marché ou des chatbots."],  
["Pour mettre en œuvre le service Azure OpenAI, il faut suivre les étapes suivantes :", "Analyser les résultats des requêtes, en utilisant les outils de suivi et de statistiques fournis par le portail Azure OpenAI."],  
["Le service Azure OpenAI repose sur deux composants principaux : le portail Azure OpenAI et les API Azure OpenAI.", "Les API Azure OpenAI sont des interfaces de programmation qui permettent d'interagir avec les modèles d'IA du service Azure OpenAI, tels que GPT-3, Codex, DALL-E ou CLIP."],  
["Les API Azure OpenAI sont des interfaces de programmation qui permettent d'interagir avec les modèles d'IA du service Azure OpenAI", "tels que GPT-3, Codex, DALL-E ou CLIP."],  
["Il permet d'accéder à des modèles d'IA de pointe, capables de réaliser des tâches complexes et variées", "telles que la génération de texte, la compréhension du langage naturel, la vision par ordinateur, la synthèse vocale ou la création de contenu multimédia."],  
["Il garantit une haute performance, en s'appuyant sur l'infrastructure cloud de Microsoft Azure", "qui assure une disponibilité, une scalabilité et une sécurité optimales."],  
["Choisir un modèle d'IA, parmi ceux proposés par le service Azure OpenAI", "ou créer son propre modèle, en utilisant les outils de formation et de réglage fournis par le service Azure OpenAI."]  
]

results = []

contexts = []

In [ ]:
for query in questions:
    result = qa_stuff({"query": query})
    results.append(result['result']) 
    sources = result["source_documents"]
    contents = [source.page_content for source in sources]  

    processed_contents = []
    for content in contents:
        if isinstance(content, str):  
            processed_content = content.split('.')[:2]  
            processed_contents.extend(processed_content)  
        else:
            print(f"Expected a string but got {type(content)}") 
    contexts.append(processed_contents)

print(len(questions))
print(len(ground_truths))
print(len(results))
print(len(contexts))

In [ ]:
# Creating a DataFrame
data = {
    'question': questions,
    'answer': results,
    'contexts': contexts,
    'ground_truth': ground_truths
}

df = pd.DataFrame(data)

# Display the DataFrame
print(df.head())

# Save as .csv
df.to_csv('ragas_aoai_dataset.csv', index=False)  

In [ ]:
# https://blog.cellenza.com/securite-2/utiliser-azure-openai-langchain-et-ragas-pour-la-classification-des-documents-confidentiels-et-la-protection-des-donnees-sensibles/ 

# from ragas.testset.generator import TestsetGenerator
# from ragas.testset.evolutions import simple, reasoning, multi_context

# testset = generator.generate_with_langchain_docs(documents, test_size=10)

# print(testset)

Evaluer


In [ ]:
# !pip install ragas datasets

In [ ]:
import pandas as pd 
from datasets import Dataset
from tqdm import tqdm
import pyarrow as py
import pyarrow as pa
import pyarrow.dataset as ds
from ragas import evaluate
from ragas.metrics import answer_relevancy, answer_similarity, answer_correctness
from ragas.metrics import context_recall, context_precision, context_entity_recall, context_utilization, context_relevancy
from ragas.metrics import faithfulness
from ragas.metrics.critique import harmfulness, maliciousness, coherence, correctness, conciseness
from ragas.llms import LangchainLLMWrapper
from datasets import Dataset, Features, Sequence, Value, load_dataset


# ragas.metrics.answer_relevancy : Scores the relevancy of the answer according to the given question.
# ragas.metrics.answer_similarity : Scores the semantic similarity of ground truth with generated answer.
# ragas.metrics.answer_correctness : Measures answer correctness compared to ground truth as a combination of factuality and semantic similarity.
# ragas.metrics.context_precision : Average Precision is a metric that evaluates whether all of the relevant items selected by the model are ranked higher or not.
# ragas.metrics.context_recall : Estimates context recall by estimating TP and FN using annotated answer and retrieved context.
# ragas.metrics.context_entity_recall : Calculates recall based on entities present in ground truth and context.

# Définir le dataset explicitement 
features = Features({
    "question": Value("string"),
    "answer": Value("string"),
    "contexts": Sequence(Value("string")),
    "ground_truth": Value("string")
})
dataset = Dataset.from_dict(df, features=features)

In [ ]:
from ragas.llms import LangchainLLMWrapper

# wrapper around azure_model 
ragas_azure_model = LangchainLLMWrapper(llm)
# patch the new RagasLLM instance
faithfulness.llm = ragas_azure_model
answer_relevancy.llm = ragas_azure_model
context_precision.llm = ragas_azure_model
context_recall.llm = ragas_azure_model
# harmfulness.llm = ragas_azure_model


In [ ]:
from langchain.embeddings import OpenAIEmbeddings

# only for answer_relevancy
azure_embeddings = OpenAIEmbeddings(
    # deployment="text-embedding-ada-002",
    model="text-embedding-ada-002",
    openai_api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    openai_api_type="azure"
)

answer_relevancy.embeddings = azure_embeddings
answer_similarity.embeddings = azure_embeddings
answer_correctness.embeddings = azure_embeddings

In [ ]:
evaluate_contexts = evaluate(
       dataset=dataset,
       metrics=[context_recall, context_precision],
       llm=ragas_azure_model,
       raise_exceptions=False,
)

evaluate_contexts

In [ ]:
evaluate_contexts_df = evaluate_contexts.to_pandas()
evaluate_contexts_df.sort_values(by='context_precision', ascending=True).head(3)

In [ ]:
evaluate_answers = evaluate(
       dataset=dataset,
       metrics=[faithfulness, answer_relevancy, answer_similarity, answer_correctness],
       llm=ragas_azure_model,
       raise_exceptions=False,
)

evaluate_answers

In [ ]:
evaluate_answers_df = evaluate_answers.to_pandas()
evaluate_answers_df.sort_values(by='faithfulness', ascending=True).head(3)

In [ ]:
evaluate_aspect_critique = evaluate(
       dataset=dataset,
       metrics=[harmfulness, maliciousness, coherence, correctness, conciseness],
       llm=ragas_azure_model,
       raise_exceptions=False,
)

evaluate_aspect_critique

In [ ]:
evaluate_aspect_critique_df = evaluate_aspect_critique.to_pandas()
evaluate_aspect_critique_df.sort_values(by='correctness', ascending=True).head(3)

### BERT score

In [ ]:
#!pip install evaluate bert_score

Currently, the best model is microsoft/deberta-xlarge-mnli, please consider using it instead of the default roberta-large in order to have the best correlation with human evaluation.

https://docs.google.com/spreadsheets/d/1RKOVpselB98Nnh_EOC4A2BYn8_201tmPODpNWu4w7xI/edit#gid=0

In [ ]:
from evaluate import load

bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")
# default model lang=en: roberta-large
results

precision: The precision for each sentence from the predictions + references lists, which ranges from 0.0 to 1.0.

recall: The recall for each sentence from the predictions + references lists, which ranges from 0.0 to 1.0.

f1: The F1 score for each sentence from the predictions + references lists, which ranges from 0.0 to 1.0.

hashcode: The hashcode of the library.

In [ ]:
# Ajuster les options pandas pour afficher les textes complets
pd.set_option('display.max_colwidth', None)  # Désactive la troncation des colonnes
pd.set_option('display.max_rows', None)      # Désactive la troncation des lignes
pd.set_option('display.max_columns', None)   # Affiche toutes les colonnes
pd.set_option('display.width', None)         # Ajuste automatiquement la largeur d'affichage


In [ ]:
predictions = answers
references = [sublist[0] for sublist in ground_truths]
results = bertscore.compute(predictions=predictions, references=references, lang="fr", model_type="microsoft/deberta-xlarge-mnli")
results

In [ ]:
data = {
    'predictions': predictions,
    'references': references,
    'precision': results['precision'],
    'recall': results['recall'],
    'f1': results['f1']
}

df = pd.DataFrame(data)

print(df.sort_values(by='precision', ascending=True).head(3))

In [ ]:
questions[4]

In [ ]:
df.iloc[4]

In [ ]:
print(df.sort_values(by='precision', ascending=False).head(3))